<a href="https://colab.research.google.com/github/farhan885/Saylani-Python-Assignment/blob/master/A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# My first CoLab
print("Hello A4")